In [1]:
import embit

from binascii import b2a_base64
from hashlib import sha256

from embit import bip32, compact, ec
from embit.bip32 import HDKey
from embit.descriptor import Descriptor
from embit.networks import NETWORKS
from embit.util import secp256k1

from embit.util.key import SECP256K1
from embit.util import key

from blockcypher import get_address_full
from blockcypher import get_transaction_details

from tinyec.ec import SubGroup, Curve
import numpy as np
import hashlib
import libnum
import bech32
import binascii

from cryptos import *

import ecdsa
from ecdsa import SigningKey, SECP256k1
import hashlib
import base58
from ecdsa.util import sigencode_der, sigdecode_der

from embit.util import secp256k1
from embit.util import py_secp256k1

import embit

from binascii import b2a_base64
from hashlib import sha256

from embit import bip32, compact, ec
from embit.bip32 import HDKey
from embit.descriptor import Descriptor
from embit.networks import NETWORKS
from embit.util import secp256k1

# LOTS OF VARIABLES ARE HARDCODED!!! GO TO THE BOTTOM FOR PVT KEY CALCULATION LOGIC!

In [2]:
def double_sha256(data):
    
    in_bytes = bytes.fromhex(data)
    
    # First SHA256 hash
    first_hash = hashlib.sha256(in_bytes).digest()

    # Second SHA256 hash
    second_hash = hashlib.sha256(first_hash).digest()

    return second_hash.hex()

In [3]:
import hmac 

SECP256K1_ORDER = n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

def deterministic_k(secret, z, extra_data=None):
    # RFC6979, optimized for secp256k1
    k = b"\x00" * 32
    v = b"\x01" * 32
    if z > SECP256K1_ORDER:
        z -= SECP256K1_ORDER
    z_bytes = z.to_bytes(32, "big")
    secret_bytes = secret.to_bytes(32, "big") #because sha256(pub_key is always in bytes)
    if extra_data is not None:
        z_bytes += extra_data
    k = hmac.new(k, v + b"\x00" + secret_bytes + z_bytes, "sha256").digest()
    v = hmac.new(k, v, "sha256").digest()
    k = hmac.new(k, v + b"\x01" + secret_bytes + z_bytes, "sha256").digest()
    v = hmac.new(k, v, "sha256").digest()
    while True:
        v = hmac.new(k, v, "sha256").digest()
        candidate = int.from_bytes(v, "big")
        if candidate >= 1 and candidate < SECP256K1_ORDER:
            return candidate
        k = hmac.new(k, v + b"\x00", "sha256").digest()
        v = hmac.new(k, v, "sha256").digest()

In [4]:
def const_k_sig(z, pvt_key, pub_hash_k = None , z_int = None):
    p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 -1                               # proven prime number  
    n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141                 # number of points in the curve/field
    a, b = 0, 7                                                                            # y^2 = x^3 + ax + b -> secpk256k1 curve 

    g_x = 55066263022277343669578718895168534326250603453777594175500187360389116729240 
    g_y = 32670510020758816978083085130507043184471273380659243275938904335757337482424
    generator_pt = (g_x, g_y)                                                              # generator point 

    #all the required functions 
    def mod_inv(a,n = p): #Extended Euclidean Algorithm/'division' in elliptic curves
        lm, hm = 1,0
        low, high = a%n,n
        while low > 1:
            ratio = high//low
            nm, new = hm-lm*ratio, high-low*ratio
            lm, low, hm, high = nm, new, lm, low
        return lm % n

    def point_double(pt):
        lam = ((3*pt[0]**2 + a) * mod_inv(2*pt[1], p)) % p
    
        x = (lam**2 - 2*pt[0]) % p 
        y = (lam*(pt[0] - x) - pt[1]) % p
    
        return (x, y)


    def point_add(pt1, pt2):
        lam = ((pt2[1] - pt1[1]) * mod_inv((pt2[0] - pt1[0]), p)) % p
    
        x = (lam**2 - pt1[0] - pt2[0]) % p
        y = (lam*(pt1[0] - x) - pt1[1]) % p 
    
        return (x, y)


    # function to produce pub key from pvt key and generator pt 
    def point_mul(generator_pt, pvt_key_hex):
        if pvt_key_hex  == 0 or  pvt_key_hex >= n:
            raise Exception("Invalid Key")
        
        # pvt key in hex to binary 
        pvt_key_bin = str(bin(pvt_key_hex))[2:]
        r = generator_pt
    
        for i in range(1, len(pvt_key_bin)):
            # point double the pt for every bit in the pvt key in binary 
            r = point_double(r)
        
            # if bit = 1, then both point double & point add
            if pvt_key_bin[i] == '1':
                r = point_add(r, generator_pt)
            
            # return this new pt 'r' which is the pub key || pub key = pvt key * generator pt       
        
        return r  

    # defining constants
    pub_key = point_mul(generator_pt, int.from_bytes(pvt_key, 'big'))    # uncompressed pub key, i.e, the product of point mul
    #print(pub_key)
    
    k = deterministic_k(pub_hash_k, z_int)
    #k = 3
    
    g = generator_pt
    x_r, y_r = point_mul(g, k)
    
    r = x_r % n
    z = int.from_bytes(z[:32], 'big')
    s = (((z + x_r*int.from_bytes(pvt_key, 'big')) % n) * mod_inv(k, n)) % n    # regular math

    return r.to_bytes(32, "little") + s.to_bytes(32, "little")

In [5]:
def ecdsa_signature_serialize_der(sig, context=None):
    if len(sig) != 64:
        raise ValueError("Signature should be 64 bytes long")
    r = int.from_bytes(sig[:32], "little")
    print("r in int :", r)
    s = int.from_bytes(sig[32:], "little")
    print("s in int :", s)
    rb = r.to_bytes((r.bit_length() + 8) // 8, "big")
    print("r in big endian :", rb)
    sb = s.to_bytes((s.bit_length() + 8) // 8, "big")
    print("s in big endian :", sb)
    return (
        b"\x30"
        + bytes([4 + len(rb) + len(sb), 2, len(rb)])
        + rb
        + bytes([2, len(sb)])
        + sb
    )

In [6]:
signature = '30440220700b0412b9ec3adebf2ef14137c3d5a4a833ca59804f57349bba3fef68fe7b0c02201440fe0c8d733a567fb527505b4ff26ba6f86acd80bf60081c831e4c9eb8628c01'

In [7]:
len(signature)

142

In [8]:
bip39_seed = '3a11298f7bf61605c6740a44aced41a547069298a0c971d7a0fd69c3c0395a21af3b6c937e700a336457edd67a6eacee6928259082304ad8adecb1a2560781a9'
bip39_seed_bytes = bytes.fromhex(bip39_seed)


embit_network = 'main'
derivation = "m/84'/0'/0'/0/7"

seed_bytes = bip39_seed_bytes
root = bip32.HDKey.from_seed(seed_bytes, version=NETWORKS[embit_network]["xprv"])
prv = root.derive(derivation).key


pub = py_secp256k1.ec_pubkey_create(prv._secret)
pub_hash_bytes = hashlib.sha256(pub).digest()




#pub_key_hash = sha256(pub)
#print(pub_hex)

prv_bytes = prv.secret
prv_int = int.from_bytes(prv_bytes, "big")


In [9]:
prv.get_public_key()

PublicKey(02581d9dc848fa707abc32329ab44caf5607583ec115664bb9b49ea562651fff71)

In [10]:
from embit import psbt
from binascii import a2b_base64, b2a_base64

psbt_tx = 'cHNidP8BAFICAAAAAfcjfDpDsaYdqQX+yFo7ulPODG/XRxaLhVzYSKtKmS2zAgAAAAD/////AewqAAAAAAAAFgAUOLzEs/SViiexuLBunToNknGdAugAAAAAAAEBHxw0AAAAAAAAFgAUTuHWUqBEj9TUqfu6Tf++7TU2URAiBgJYHZ3ISPpwerwyMpq0TK9WB1g+wRVmS7m0nqViZR//cRi+E/8RVAAAgAAAAIAAAACAAAAAAAcAAAAAAA=='

raw = a2b_base64(psbt_tx)
tx = psbt.PSBT.parse(raw)

total_in = 0
for inp in tx.inputs:
    total_in += inp.witness_utxo.value
print("Inputs:", total_in, "satoshi")

change_out = 0  # value that goes back to us
send_outputs = []
for i, out in enumerate(tx.outputs):
    # check if it is a change or not:
    change = False
    # should be one or zero for single-key addresses
    for pub in out.bip32_derivations:
        # check if it is our key
        if out.bip32_derivations[pub].fingerprint == fingerprint:
            hdkey = root.derive(out.bip32_derivations[pub].derivation)
            mypub = hdkey.key.get_public_key()
            if mypub != pub:
                raise ValueError("Derivation path doesn't look right")
                
            # now check if provided scriptpubkey matches
            sc = script.p2wpkh(mypub)
            if sc == tx.tx.vout[i].script_pubkey:
                change = True
                continue
                
    if change:
        change_out += tx.tx.vout[i].value
    else:
        send_outputs.append(tx.tx.vout[i])
    print("Spending", total_in - change_out, "satoshi")
    
    fee = total_in - change_out
    for out in send_outputs:
        fee -= out.value
        print(out.value, "to", out.script_pubkey.address(NETWORKS["main"]))
    print("Fee:", fee, "satoshi")



Inputs: 13340 satoshi
Spending 13340 satoshi
10988 to bc1q8z7vfvl5jk9z0vdckphf6wsdjfce6qhgskp84e
Fee: 2352 satoshi


In [150]:
tx.tx

Transaction(0200000001f7237c3a43b1a61da905fec85a3bba53ce0c6fd747168b855cd848ab4a992db30200000000ffffffff01ec2a00000000000016001438bcc4b3f4958a27b1b8b06e9d3a0d92719d02e800000000)

In [17]:
txn_details_hex = '0200000001f7237c3a43b1a61da905fec85a3bba53ce0c6fd747168b855cd848ab4a992db30200000000ffffffff01ec2a00000000000016001438bcc4b3f4958a27b1b8b06e9d3a0d92719d02e800000000'

In [15]:
# 18/1/24
txn_details_hex  = '02000000010a0befb5ef7dbc1b7022788f200ca6606ca65024e1a4a499f1d4a38de1ef01880100000000ffffffff01b927000000000000160014ffa40104eec28c18911c803f1207f8fb6d993fc800000000'

In [11]:
len('0200000001f7237c3a43b1a61da905fec85a3bba53ce0c6fd747168b855cd848ab4a992db30200000000ffffffff016423000000000000160014ffa40104eec28c18911c803f1207f8fb6d993fc800000000')

164

In [23]:
# 19/1/24
txn_details_hex = '0200000001f7237c3a43b1a61da905fec85a3bba53ce0c6fd747168b855cd848ab4a992db30200000000ffffffff016423000000000000160014ffa40104eec28c18911c803f1207f8fb6d993fc800000000'

In [13]:
len(txn_details_hex)

164

In [24]:
print(txn_details_hex[0:8])
print(txn_details_hex[8:10])
print(txn_details_hex[10:74])
print(txn_details_hex[74:82])
print(txn_details_hex[82:84])
print(txn_details_hex[84:92])

print('\n')
print(txn_details_hex[92:94])
print(txn_details_hex[94:110])   # output amount 
print(txn_details_hex[110:112])
print(txn_details_hex[112:156])
print(txn_details_hex[156:164])

02000000
01
f7237c3a43b1a61da905fec85a3bba53ce0c6fd747168b855cd848ab4a992db3
02000000
00
ffffffff


01
6423000000000000
16
0014ffa40104eec28c18911c803f1207f8fb6d993fc8
00000000


In [275]:
len('0014ffa40104eec28c18911c803f1207f8fb6d993fc8')

44

txn: https://mempool.space/tx/d882925ae657bcf2ee252ccfdffdf46fa87309168099af20727ae35833267414

In [27]:
#19/ 1/ 24 -> for above txn 
ver = txn_details_hex[0:8]
prevouts = txn_details_hex[10:74] + txn_details_hex[74:82]
seq = txn_details_hex[84:92]
outpoint = prevouts

#scriptcode = '1976a914' + '5d50044454d3d18006e71b0da30d5c39b793053f' + '88ac'
scriptcode = '1976a914' + '4ee1d652a0448fd4d4a9fbba4dffbeed35365110' + '88ac' # addr -> bc1qfmsav54qgj8af49flwaymla7a56nv5gszss2u7
#amount = '1935000000000000'  # input val = 13593 satoshi
amount = '1c34000000000000' # input val = 13340

output = txn_details_hex[94:110] + txn_details_hex[110:112] + txn_details_hex[112:156]
locktime = '00000000'
hashtype = '01000000'

In [26]:
# old txn , do not run
ver = txn_details_hex[0:8]
prevouts = txn_details_hex[10:74] + txn_details_hex[74:82]
seq = txn_details_hex[84:92]
outpoint = prevouts
scriptcode = '1976a914' + '4ee1d652a0448fd4d4a9fbba4dffbeed35365110' + '88ac'
amount = '1c34000000000000'  # input val = 13340 satoshi
output = txn_details_hex[94:110] + txn_details_hex[110:112] + txn_details_hex[112:156]
locktime = '00000000'
hashtype = '01000000'

In [28]:
prevouts_hash = double_sha256(prevouts)
print("prevouts double hash :", prevouts_hash)
seq_hash = double_sha256(seq)
print("seq double hash :", seq_hash)
outputs_hash = double_sha256(output)
print("outputs double hash :", outputs_hash)

prevouts double hash : 5600a8e58a8f4abe6265c31a13b42fda89772d4b7c953d6d39994de9ffb3927f
seq double hash : 3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044
outputs double hash : 32549df5abff5731231eaad1d373b5808b59820b3c4281cc0dbadc296c6f9470


In [29]:
pre_image = ver + prevouts_hash + seq_hash + outpoint + scriptcode + amount + seq + outputs_hash + locktime + hashtype

In [30]:
len(pre_image)

364

In [55]:
pre_image

'020000005600a8e58a8f4abe6265c31a13b42fda89772d4b7c953d6d39994de9ffb3927f3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044f7237c3a43b1a61da905fec85a3bba53ce0c6fd747168b855cd848ab4a992db3020000001976a9144ee1d652a0448fd4d4a9fbba4dffbeed3536511088ac1c34000000000000ffffffff32549df5abff5731231eaad1d373b5808b59820b3c4281cc0dbadc296c6f94700000000001000000'

In [230]:
# testing new txn 18/1/24 , dont run cell 
pre_image = '0200000006f2b86854f8d3e7d86cb1e39c2b09919adde421fed7a141e6515ffd7f88a6843bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b000000001976a91438bcc4b3f4958a27b1b8b06e9d3a0d92719d02e888ac3d53000000000000ffffffff9e573421d9f32fc4f9997b707ddac0dd74d506a5295ab4c57af4608e29a0ae180000000001000000'

pre_image_hash_to_sign = double_sha256(pre_image)
pre_image_hash_to_sign_bytes = bytes.fromhex(pre_image_hash_to_sign)

msghash = pre_image_hash_to_sign_bytes
msghash_int = int.from_bytes(msghash, "big")


pub_hash_int = 38933762198836990009719519817402630601322716143883358142998450627002174767517

signature = '304402201626d2d8533e6f3f9150a88e565068037e012ba156ebd376666f05a9da1fb7e702206b37258e900391bc1f104ee4556830730a7d0291c5fb8f52eb15bcbbe9edbd6201'


# everything below is old and working 

#pre_image_hash_to_sign = double_sha256(pre_image) -> original code



#pre_image_hash_to_sign_bytes = bytes.fromhex(pre_image_hash_to_sign)

#msghash = pre_image_hash_to_sign_bytes
#msghash_int = int.from_bytes(msghash, "big")

In [41]:
pre_image_hash_to_sign = double_sha256(pre_image) 

pre_image_hash_to_sign_bytes = bytes.fromhex(pre_image_hash_to_sign)

msghash = pre_image_hash_to_sign_bytes
msghash_int = int.from_bytes(msghash, "big")

pub_hash_int = 11213424163078702846961618719436802557356559473721032386514974657038528928268

signature = '304402203193c5f490cb13e4c3cb2f6cbbb4d0c6af632ff8efa7ef12dd8bc5249dc5f50f0220334bdf2a3db5f98ae54ddb4bf90e83ff0874c51ed5a3a6b10ad3fddd94ec396f01'

prv_hex = 'ce1539c8fe4a767d3a82329e806493c8034cd946371103223353ca82605ec51c'

prv_int = int.from_bytes(bytes.fromhex(prv_hex), "big")

#  Private key calculation logic 

In [42]:
p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 -1 
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

def mod_inv(a,n = p): #Extended Euclidean Algorithm/'division' in elliptic curves
    lm, hm = 1,0
    low, high = a%n,n
    
    while low > 1:
        ratio = high//low
        nm, new = hm-lm*ratio, high-low*ratio
        lm, low, hm, high = nm, new, lm, low
        
    return lm % n

In [43]:
r_len = 2*int(signature[6:8], 16)
r_hex = signature[8:(8+r_len)]

s_len = 2*int(signature[(8+r_len+2):(8+r_len+2+2)], 16)
s_hex = signature[(8+r_len+2+2):(8+r_len+2+2+s_len)]

In [47]:
r_int = int(r_hex, 16)
s_int = int(s_hex, 16)

#k = deterministic_k(prv_int, msghash_int)

k = deterministic_k(pub_hash_int, msghash_int) 


81315389345406280729888218297120106925287080885957500133984651971236021306488


In [48]:
# to calculate pvt. key we already know r, s and k 
calc_prv = (mod_inv(r_int, n)*(k*s_int - msghash_int)) % n 

print(hex(calc_prv))

0xce1539c8fe4a767d3a82329e806493c8034cd946371103223353ca82605ec51c


In [54]:
hex(prv_int) == hex(calc_prv)

True